<a href="https://colab.research.google.com/github/AlinaZakharova1997/miramedix_practice/blob/main/%D0%A3%D0%BB%D1%83%D1%87%D1%88%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2%D1%8B%D0%B1%D0%BE%D1%80%D0%BA%D0%B8_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85_%D0%B4%D0%BB%D1%8F_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Изначально было собрано 10000 текстов, из них 3303 оказались дубликатами. Поэтому пришлось пересобрать файлы train, val, test.
Выгрузили их с диска, собрали в один список all_data, это получился список вложенных словарей, которые преобразовали в просто список словарей new_data, чтобы получилось сделать из него датафрейм df. Из датафрейма с помощью функции drop_duplicates() были автоматически удалены все дубликаты. В итоге получилось 6697 текстов. Их поделили заново на три части train, val, test в том же соотношении 0,5:0,25:0,25 и сформировали новые файлы jsonl для обучения и проверки модели.

In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
import json
import pandas as pd 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
all_data = []

with jsonlines.open('/content/drive/MyDrive/test_data.json', 'r') as reader:
  for obj in reader:
    if obj not in all_data:
      all_data.append(obj)
  
with jsonlines.open('/content/drive/MyDrive/val_data.json', 'r') as reader:
  for obj in reader:
    if obj not in all_data:
      all_data.append(obj)

with jsonlines.open('/content/drive/MyDrive/train_data.json', 'r') as reader:
  for obj in reader:
    if obj not in all_data:
      all_data.append(obj)

In [ ]:
new_data = []
for item in all_data:
  pairs = {}
  pairs['de'] = item['translation']['de']
  pairs['en'] = item['translation']['en']
  new_data.append(pairs)

In [ ]:
df = pd.DataFrame(new_data)

In [ ]:
df = df.drop_duplicates(keep='first')

In [ ]:
data = []
for i in range(len(df)):
  translation = {}
  langs = {}
  langs['de'] = df['de'][i]
  langs['en'] = df['en'][i]
  translation['translation'] = langs
  #print(langs)
  data.append(translation)

In [ ]:
train = data[:3347]
val = data[3347:5020]
test = data[5020:]

In [ ]:
with jsonlines.open('test_data.jsonl', mode='w') as writer:
    writer.write_all(test)
with jsonlines.open('val_data.jsonl', mode='w') as writer:
    writer.write_all(val)
with jsonlines.open('train_data.jsonl', mode='w') as writer:
    writer.write_all(train)